In [1]:
import numpy as np
from utils import temporal_graph_load
from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

In [2]:
net = 'DTU'
#net = 'synth'

In [3]:
eps_I_list = [0.25]
eps_T_list = [0.1]
PARAMETERS = {'temporal_gap' : 300,           # Temporal gap between static networks (sec)
              'nb_clusters' : 10,                   # Nb of clusters in networks
              'cluster_size' : 67,                  # Cluster size
              'nb_links' : 91,                    # nb of links in networks
              'memory_contacts' : 7,                # Tracing memory (days)
              'max_time_quar' : 10,                 # Quarantine duration (days)
              'max_time_iso' : 30,                  # Isolation duration (days)
              'eps_Is' : eps_I_list,                # Isolation effectivity
              'eps_Ts' : eps_T_list,                # Tracing effectivity
              'lam_k' : 0.002,                      # Knowledge spreading parameter
              'times' : 40,                        # Number of repetition of the simulation
              'seed' : 5,                          # Random seed 
              'Y_i' : 1,                            # Number of initially infected people
              'nc' : 0.0,                             # Fraction of individuals who are not adopting the app
              'symptomatics' : 0.8,                 # Fraction of symptomatic individuals 
              'testing' : 0.0,                      # Fraction of asymptomatics who are detected via random testing
              'net' : net,
              'store':{'to_store': True,            # Save the results?
                       'path_to_store':'RESULTS/'}} # Target folder to save the results

In [4]:
def w_mean(vector,weights):
    num = 0
    if len(vector) != len(weights):
        print('error')
    else:
        for i in range(len(vector)):
            num += vector[i]*weights[i]
        den = sum(weights)
        res = num/den
        return res

## Prova su reti sintetiche 

In [5]:
net = 'synth'

In [6]:
nb_clusters = PARAMETERS['nb_clusters']
cluster_size = PARAMETERS['cluster_size']
L = PARAMETERS['nb_links']

In [7]:
#for p_intra in [0.00179,0.00229,0.00277,0.00317,0.00348,0.00370,0.00385,0.00395,0.00401]:
for p_intra in [0.00317]:
    p_inter = 2*L/(nb_clusters*(nb_clusters-1)*cluster_size**2) - (cluster_size-1)*p_intra/((nb_clusters-1)*cluster_size)
    graph_name = 'temporal_net_p_intra%.2E_p_inter%.2E'%(p_intra,p_inter)
    graphs = temporal_graph_load(graph_name)

Compute nb of links:

In [8]:
nb_links = 0
for n in range(len(graphs)):
    nb_links += len(graphs[n].edges())
nb_links = nb_links/len(graphs)

In [9]:
nb_links

91.071475

Compute $p_{intra}$ and $p_{inter}$:

In [10]:
intra_cluster_links = []
extra_cluster_links = []
for n in range(nb_clusters):
    intra_cluster_n = 0
    extra_cluster_n = 0
    for node in range(n*cluster_size, (n+1)*cluster_size):
        for graph in graphs:
            neighs = graph.neighbors(node)
            for neigh_node in neighs:
                if n*cluster_size <= neigh_node < (n+1)*cluster_size:
                    intra_cluster_n += 1
                else:
                    extra_cluster_n += 1
    intra_cluster_n /= 2
    extra_cluster_n /= 2
    
    intra_cluster_links.append(intra_cluster_n)
    extra_cluster_links.append(extra_cluster_n)
    n += 1

intra_links = np.mean(intra_cluster_links) # media sui diversi cluster
inter_links = np.mean(extra_cluster_links)
      

In [11]:
estimated_p_intra = intra_links/(len(graphs)*cluster_size*(cluster_size-1)/2)
print('estimated p_intra',estimated_p_intra,' real p_intra',p_intra)

estimated p_intra 0.003172473993668023  real p_intra 0.00317


In [12]:
nb_nodes = cluster_size*PARAMETERS['nb_clusters']
estimated_p_inter = inter_links/(len(graphs)*cluster_size*(nb_nodes - cluster_size)/2)
print('estimated p_inter',estimated_p_inter,' real p_inter',p_inter)

estimated p_inter 0.00010360176728298805  real p_inter 0.00010351872478403995


Funziona!

## Calcolo su DTU

In [13]:
net = 'DTU'

In [14]:
graph_name = 'DTU_temporal_gap_%d_rssi_filter_-80_relabeled'%PARAMETERS['temporal_gap']
#p_intra  = 0.00277
#p_inter = 2.7282800584048545e-05
#graph_name = 'prova_DTU_p_intra%.2E_p_inter%.2E'%(p_intra,p_inter)
graphs = temporal_graph_load(graph_name)

Compute nb of links:

In [15]:
nb_links = 0
for n in range(len(graphs)):
    nb_links += len(graphs[n].edges())
nb_links = nb_links/len(graphs)

In [16]:
nb_links

90.71958328165695

In [17]:
part_d = np.load('partition_d_10clusters.npy',allow_pickle=True)[0]

In [18]:
sizes = [0 for n in range(max(part_d.values())+1)]
for node in part_d.keys():
    sizes[part_d[node]] += 1
sizes

[57, 102, 45, 55, 135, 99, 106, 67, 7, 2]

In [19]:
nb_clusters = len(sizes)
clusters = []
for n in range(nb_clusters):
    clusters.append([k for k,v in part_d.items() if v == n])

Compute $p_{intra}$ and $p_{inter}$:

In [20]:
intra_cluster_links_DTU = []
extra_cluster_links_DTU = []
for n in range(nb_clusters):
    intra_cluster_n = 0
    extra_cluster_n = 0
    for node in clusters[n]:
        for graph in graphs:
            if node in graph:
                neighs = graph.neighbors(node)
                for neigh_node in neighs:
                    if neigh_node in clusters[n]:
                        intra_cluster_n += 1
                    else:
                        extra_cluster_n += 1
    intra_cluster_n /= 2
    extra_cluster_n /= 2
    
    intra_cluster_links_DTU.append(intra_cluster_n)
    extra_cluster_links_DTU.append(extra_cluster_n)
    n += 1

#intra_links = np.mean(intra_cluster_links_DTU) # media sui diversi cluster
#inter_links = np.mean(extra_cluster_links_DTU)
      

In [21]:
p_intra_clusters = np.zeros(nb_clusters)
for n in range(nb_clusters):
    p_intra_clusters[n] = intra_cluster_links_DTU[n]/(len(graphs)*sizes[n]*(sizes[n]-1)/2)
p_intra_clusters

array([0.0044092 , 0.002045  , 0.00435034, 0.00367777, 0.00181729,
       0.00174481, 0.00166697, 0.0031274 , 0.03563603, 0.60300136])

In [22]:
w = [n*(n-1) for n in sizes]
estimated_p_intra = w_mean(p_intra_clusters,w)
print('estimated p_intra',estimated_p_intra)#,'real p_intra',p_intra)

estimated p_intra 0.002282698393185423


In [23]:
nb_nodes = 675

p_inter_clusters = np.zeros(nb_clusters)
for n in range(nb_clusters):
    p_inter_clusters[n] = extra_cluster_links_DTU[n]/(len(graphs)*sizes[n]*(nb_nodes - sizes[n])/2)
p_inter_clusters

array([6.96940200e-05, 1.10037112e-04, 1.10969151e-04, 1.22084770e-04,
       1.34712459e-04, 8.99464835e-05, 1.23712816e-04, 7.15167837e-05,
       6.40009116e-05, 3.94368346e-05])

In [24]:
w = [n*(n-1) for n in sizes]
estimated_p_inter = w_mean(p_inter_clusters,w)
print('estimated p_inter',estimated_p_inter)#,'real p_inter',p_inter)

estimated p_inter 0.00011224904144768468


Verifiche:

In [31]:
#import matplotlib.pyplot as plt
#plt.plot(sizes,'+') 


In [32]:
#plt.plot(p_intra_clusters, 'o')
#plt.plot(p_inter_clusters,'s')

In [33]:
#N = len(part_d)
#term1 = 0
#term2 = 0
#for i in range(len(sizes)):
#    term1 += sizes[i]*(sizes[i] - 1)*estimated_p_intra/2
#    term2 += sizes[i]*(N - sizes[i])*estimated_p_inter/2

#term1 + term2
